### cycle in graph 
    Union-find => O(n)
    Union-rank => O(logn)

In [5]:
# union-find
from collections import defaultdict

class Graph:
    def __init__(self, vertices):
        self.vertices = vertices
        self.graph = defaultdict(list)
        
    def addEdge(self, x, y):
        self.graph[x].append(y)
        
    def findParent(self, parent, i):
        if parent[i] == -1:
            return i
        else:
            return self.findParent(parent, parent[i])
        
    def union(self, parent, x, y):
        x_parent = self.findParent(parent, x)
        y_parent = self.findParent(parent, y)
        parent[x_parent] = y_parent


def detect_cycle(g, vertices):
    parent = [-1 for i in range(vertices)]
    graph = g.graph

    for vertex in range(vertices):
        for node in graph[vertex]:
            if node >= vertices or vertex >= vertices:
                return -1
            v_parent = g.findParent(parent, vertex)
            n_parent = g.findParent(parent, node)
            print(vertex, node, "%", v_parent, n_parent, 'p', parent)
            if v_parent == n_parent:
                return True
            g.union(parent, v_parent, n_parent)
    return False


n = 4
g = Graph(n)
g.addEdge(0, 1)
g.addEdge(1, 2)
g.addEdge(2, 3)
g.addEdge(3, 0)
detect_cycle(g, n)


0 1 % 0 1 p [-1, -1, -1, -1]
1 2 % 1 2 p [1, -1, -1, -1]
2 3 % 2 3 p [1, 2, -1, -1]
3 0 % 3 3 p [1, 2, 3, -1]


True

### union rank by rank and path compression

In [17]:
from collections import defaultdict

class Subset:
    def __init__(self, parent, rank):
        self.parent = parent
        self.rank = rank
        
class Graph:
    def __init__(self, vertices):
        self.vertices = vertices
        self.graph = defaultdict(list)
        
    def addEdge(self, x, y):
        self.graph[x].append(y)
        
    def findParent(self, subsets, node):
        if subsets[node].parent == -1:
            return subsets[node].parent
        else:
            subsets[node].parent = self.findParent(subsets, node)
    
    def union(self, subsets, x, y):
        x_parent = self.findParent(self, subsets, x)
        y_parent = self.findParent(self, subsets, y)
        
        if subsets[x_parent].rank < subsets[y_parent].rank:
            subsets[x_parent].parent = y_parent
        elif subsets[x_parent].rank > subsets[y_parent].rank:
            subsets[y_parent].rank = x_parent
        else:
            subsets[x_parent].rank = y_parent
            subsets[y_parent].rank += 1

    
def detect_cycle_rank_pc(g, vertices):
#     parent = [-1 for i in range(vertices)]
#     rank = [0 for i in range(vertices)]

    subsets = []
    for i in range(vertices):
        subsets.append(Subset(-1, 0))
    graph = g.graph

    for vertex in range(vertices):
        for node in graph[vertex]:
            if node >= vertices or vertex >= vertices:
                return -1
            v_parent = g.findParent(subsets, vertex)
            n_parent = g.findParent(subsets, node)
            if v_parent == n_parent:
                return True
            g.union(subsets, v_parent, n_parent)
    return False

n = 4
g = Graph(n)
g.addEdge(0, 1)
g.addEdge(1, 2)
g.addEdge(2, 3)
# g.addEdge(3, 0)
detect_cycle_rank_pc(g, n) 
    

True

In [12]:
# union-rank

from collections import defaultdict

class GraphR:
    def __init__(self, vertices):
        self.vertices = vertices
        self.graph = defaultdict(list)
        
    def addEdge(self, x, y):
        self.graph[x].append(y)
        
    def findParent(self, parent, i):
        if parent[i] == -1:
            return i
        else:
            return self.findParent(parent, parent[i])
        
        
    def union(self, parent, rank, x, y):
        x_parent = self.findParent(parent, x)
        y_parent = self.findParent(parent, y)
        
        # new parent is one with higher rank
        if rank[x_parent] < rank[y_parent]:
            parent[x_parent] = y_parent
        elif rank[x_parent] > rank[y_parent]:
            parent[y_parent] = x_parent
        else:
            parent[y_parent] = x_parent
            rank[x_parent] += 1


def detect_cycle_via_rank(g, vertices):
    parent = [-1 for i in range(vertices)]
    rank = [0 for i in range(vertices)]
    
    graph = g.graph

    for vertex in range(vertices):
        for node in graph[vertex]:
            if node >= vertices or vertex >= vertices:
                return False
            v_parent = g.findParent(parent, vertex)
            n_parent = g.findParent(parent, node)
            print(vertex, node, "%", v_parent, n_parent, 'p', parent, 'r', rank)
            if v_parent == n_parent:
                return True
            g.union(parent, rank, v_parent, n_parent)
    return False


n = 4
g = GraphR(n)
g.addEdge(0, 1)
g.addEdge(1, 2)
g.addEdge(2, 3)
g.addEdge(3, 0)

detect_cycle_via_rank(g, n)


0 1 % 0 1 p [-1, -1, -1, -1] r [0, 0, 0, 0]
1 2 % 0 2 p [-1, 0, -1, -1] r [1, 0, 0, 0]
2 3 % 0 3 p [-1, 0, 0, -1] r [1, 0, 0, 0]


False

### kruskal
    O(Elogv) or O(ElogE)
    edge sorting : O(ElogE)
    union-rank : O(logV)
    loop: O(E)
    
    E can be atmost V^2, so logE == 2logV
    total: O(ElogE) +  O(E)*O(logV)
           => O(ElogE) +  O(ElogV)
          or, O(Elogv) or O(ElogE

In [19]:
from collections import defaultdict

class Graph:
    def __init__(self, vertices):
        self.vertices = vertices
        self.graph = []
        
    def addEdge(self, x, y, w):
        self.graph.append([x, y, w])
        
    def findParent(self, parent, i):
        if parent[i] == -1:
            return i
        else:
            return self.findParent(parent, parent[i])
        
        
    def union(self, parent, rank, x, y):
        x_parent = self.findParent(parent, x)
        y_parent = self.findParent(parent, y)
        
        # new parent is one with higher rank
        if rank[x_parent] < rank[y_parent]:
            parent[x_parent] = y_parent
        elif rank[x_parent] > rank[y_parent]:
            parent[y_parent] = x_parent
        else:
            parent[y_parent] = x_parent
            rank[x_parent] += 1


def kruskal(g, vertices):
    parent = [-1 for i in range(vertices)]
    rank = [0 for i in range(vertices)]
    res = []
    edge = 0
    vertex = 0
    sortedGraph = list(sorted(g.graph, key = lambda x: x[2] ))
    print(sortedGraph)
    # for MST, vertices-1 edges possible.
    while edge < vertices - 1:
        x, y, w = sortedGraph[vertex]    
        x_parent = g.findParent(parent, x)
        y_parent = g.findParent(parent, y)
        
        if x_parent != y_parent:
            edge += 1
            res.append([x, y, w])
            g.union(parent, rank, x, y)
        vertex += 1
    print(res)
    
    
n = 4
g = Graph(n)
g.addEdge(0, 1, 10)
g.addEdge(0, 2, 7)
g.addEdge(0, 3, 4)
g.addEdge(1, 3, 15)
g.addEdge(2, 3, 3)
kruskal(g, n)


[[2, 3, 3], [0, 3, 4], [0, 2, 7], [0, 1, 10], [1, 3, 15]]
[[2, 3, 3], [0, 3, 4], [0, 1, 10]]


### prim's algorithm

In [ ]:
from collections import defaultdict

class Graph:
    def __init__(self, vertices):
        self.vertices = vertices
        self.graph = []
        
    def addEdge(self, x, y, w):
        self.graph.append([x, y, w])
        
    def findParent(self, parent, i):
        if parent[i] == -1:
            return i
        else:
            return self.findParent(parent, parent[i])
        
        
    def union(self, parent, rank, x, y):
        x_parent = self.findParent(parent, x)
        y_parent = self.findParent(parent, y)
        
        # new parent is one with higher rank
        if rank[x_parent] < rank[y_parent]:
            parent[x_parent] = y_parent
        elif rank[x_parent] > rank[y_parent]:
            parent[y_parent] = x_parent
        else:
            parent[y_parent] = x_parent
            rank[x_parent] += 1

### activity selection

In [67]:
def activity_selection(start, end, n):
    times = list(zip(start, end))
    times = list(sorted(times, key = lambda x: x[1]))
    count = 1

    last = 0
    for i in range(1, len(times)):
        if times[i][0] >= times[last][1]:
            last = i
            count += 1
    return count

n = 6
start = [1, 3, 2, 5, 8, 5]
end =  [2, 4, 6, 7, 9, 9]
activity_selection(start, end, n)


4

### page faults in LRU

In [4]:
def page_faults_in_lru(pages, cap, n):
    if cap >= n:
        return len(set(pages))

    q = []
    s = set()
    i = 0
    while len(s) < cap and i < n:
        if pages[i] in s:
            page_ind = q.index(pages[i])
            q.pop(page_ind)
        q.append(pages[i])
        s.add(pages[i])
        i += 1
    count = len(s)
    
    while i < n:
        if pages[i] not in s:
            first_page = q.pop(0)
            if first_page in s:
                s.remove(first_page)
            s.add(pages[i])
            count += 1
        else:
            page_ind = q.index(pages[i])
            q.pop(page_ind)
        q.append(pages[i])
        i += 1
    return count

n = 9
pages = [5, 0, 1, 3, 2, 4, 1, 0, 5]
cap = 4
print(page_faults_in_lru(pages, cap, n))


8


### coin piles

In [1]:
from sys import maxsize

def coin_piles_v1(piles, n, k):
    piles.sort()
    removals = maxsize
    x = 0
    y = 0
    i = 0
    
    while i < n and y < removals:
        x = y
        print("Asd", x, y, piles[i])
        for j in range(i+1, n):
            if piles[j] - piles[i] > k:
                x += piles[j] - piles[i] - k
        print(piles[i], removals, y, x)
        removals = min(removals, x)
        y += piles[i]
        i += 1
            
    return removals


n, k = 6, 3
piles = [1, 2, 5, 5, 1, 9]
print(coin_piles_v1(piles, n, k))


Asd 0 0 1
1 9223372036854775807 0 7
Asd 1 1 1
1 7 1 8
Asd 2 2 2
2 7 2 6
Asd 4 4 5
5 6 4 5
5


### max-toys

In [3]:
def max_toys(arr, n, k):
    arr.sort()
    sum = 0
    count = 0
    for i in range(n):
        if sum > k:
            break
        sum += arr[i]
        count += 1
    return count - 1

n, k = 7, 50
arr = [1, 12, 5, 111, 200, 1000, 10]
print(max_toys(arr, n, k))


4
